In [2]:
!pip install keras
!pip install tensorflow
!pip install opencv-python
import cv2
import numpy as np
from keras.models import load_model
import keras
from keras.models import model_from_json

    100% |████████████████████████████████| 317kB 16.7MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 16.4MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 19.8MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 92.5MB 557kB/s eta 0:00:01    41% |█████████████▏                  | 38.0MB 37.1MB/s eta 0:00:02
    100% |████████████████████████████████| 10.8MB 5.7MB/s eta 0:00:01
    100% |████████████████████████████████| 1.2MB 28.5MB/s ta 0:00:01
    100% |████████████████████████████████| 368kB 36.4MB/s ta 0:00:01
    100% |████████████████████████████████| 102kB 37.0MB/s a 0:00:01
    100% |████████████████████████████████| 3.2MB 16.6MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 36.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /home/ec2-u

Using TensorFlow backend.


In [3]:
json_file = open('selfie_model/selfie_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
loaded_model.load_weights('selfie_model/selfie_model-weights.h5')
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [9]:
im = cv2.imread("Positive-(11).jpg")
im_r = cv2.resize(im, (256,256))
im_r = cv2.cvtColor(im_r, cv2.COLOR_BGR2GRAY)
im_t = im_r.reshape(1,256,256,1).astype('float32') / 255

In [10]:
prediction = loaded_model.predict(im_t)

In [11]:
prediction

array([[0.01097437, 0.98902565]], dtype=float32)

In [12]:
from keras.preprocessing import image



In [18]:
imagePath = "negative-(620).jpg"
test_image = image.load_img(imagePath, target_size = (256,256), color_mode='grayscale') 
test_image = image.img_to_array(test_image)
#test_image = np.expand_dims(test_image, axis = 0)

#images = np.vstack([test_image])
im_t = test_image.reshape(1, 256,256, 1)
classes = loaded_model.predict_classes(im_t, batch_size=50)

In [46]:
classes

array([1])

In [28]:
import boto3
import boto3

client = boto3.client('s3') #low-level functional API

resource = boto3.resource('s3') #high-level object-oriented API
my_bucket = resource.Bucket('hara-ds-datascience') #subsitute this for your s3 bucket name. 

In [22]:
bucket_data = 'hara-ds-datascience'
path_test = 'Robert/image-augmentation/Test/Positive-(1001).jpg'
folder_path = 'Robert/image-augmentation/Train1/'
test_path = 'Robert/image-augmentation/Test/'
name_files = 'negative-(1056).pg'
key = folder_path + name_files
type(bucket_data)

str

In [23]:
from PIL import Image
import numpy as np
IMG_SIZE = 256

In [24]:
import os
import numpy as np
#heights = []

In [25]:
def label_image(name):
    word_label = name.split('-')[0]
    if word_label == 'Positive': return np.array([1,0])
    elif word_label == 'negative': return np.array([0,1])

In [26]:
from random import shuffle

In [29]:
files1 = list(my_bucket.objects.filter(Prefix=test_path))
files1.pop(0)
#files1

s3.ObjectSummary(bucket_name='hara-ds-datascience', key='Robert/image-augmentation/Test/')

In [30]:
def load_test_data():
    test_data = []
    for file in files1:
        nama = str(file)
        nama = nama.split(',')[1].replace("')","").replace("key='","").split("/")[3]
        label = label_image(nama)
        key = test_path + nama
        object_1 = client.get_object(Bucket=bucket_data, Key=key)
        img = Image.open(object_1['Body'])
        img = img.convert('L')
        img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        test_data.append([np.array(img), label])
    shuffle(test_data)
    return test_data

In [31]:
test_data1 = load_test_data()

In [33]:
testImages = np.array([i[0] for i in test_data1]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
testLabels = np.array([i[1] for i in test_data1])

loss, acc = loaded_model.evaluate(testImages, testLabels, verbose = 2)
print(acc * 100)

94.20970266040689


In [34]:
predictions = loaded_model.predict(testImages)

In [35]:
from sklearn.metrics import confusion_matrix
image_pred = (predictions > 0.8)
confusion_matrix(testLabels.argmax(axis=1), image_pred.argmax(axis=1))

array([[204,  11],
       [ 18, 406]])

In [36]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(testLabels.argmax(axis=1), image_pred.argmax(axis=1))

In [37]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.utils.fixes import signature
precision, recall, _ = precision_recall_curve(testLabels.argmax(axis=1), image_pred.argmax(axis=1))

In [38]:
precision_recall_curve(testLabels.argmax(axis=1), image_pred.argmax(axis=1))

(array([0.66353678, 0.9736211 , 1.        ]),
 array([1.        , 0.95754717, 0.        ]),
 array([0, 1]))

In [39]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [40]:
roc_auc_score(testLabels.argmax(axis=1), image_pred.argmax(axis=1))

0.9531921895568232

In [41]:
f1_score(testLabels.argmax(axis=1), image_pred.argmax(axis=1))

0.9655172413793104

In [42]:
precision_score(testLabels.argmax(axis=1), image_pred.argmax(axis=1))

0.973621103117506

In [43]:
recall_score(testLabels.argmax(axis=1), image_pred.argmax(axis=1))

0.9575471698113207

In [44]:
accuracy_score(testLabels.argmax(axis=1), image_pred.argmax(axis=1))

0.9546165884194053